# SciAuth Tutorial

In this tutorial you will learn how to:
1. Create a SciToken signing key.
2. Publish the key to a website to create an issuer.
3. Use a token to submit to a HTCondor node.

### Requirements
- GitHub account
- This notebook already has the python-scitokens package installed.  You can install it with `pip install scitokens`

This page is a special `bash` type page rather than Python.  Commands entered into this page are executed on the command line, not within a python interpreter.

## Creating a Key

We first will create a private and public key to sign tokens that we will create.

Create the private key with the `scitokens-admin-create-key` program.

In [ ]:
scitokens-admin-create-key --create-keys --pem-private > private.pem

Take a look at the newly created private key, `private.pem`.  You may want to **backup** the file since it's an important private key.  Feel free to download it from the file browser at the left.

In [ ]:
cat private.pem

Next we will create the public key, in jwks format so that we can upload it to the website.

In [ ]:
scitokens-admin-create-key --private-keyfile private.pem --jwks-public

In order for an issuer to work, you need:
1. A metadata page for clients to find the keys (`username/.well-known/openid-configuration`) 
2. The public keys

The metadata page is very short with only 2 attributes.  In the example below, be sure to replace my github username (djw8605) with your own.

Steps to create the issuer:
1. Create the configuration file by going to the [repo website](https://github.com/SciAuth/htcondor-week-2022), clicking on "Create a new file, then enter your github **username**, then "/", then `.well-known`, then "/", then `openid-configuration`

    Metadata page (`openid-configuration`):
    ```json
    {  
        "issuer":"https://sciauth.org/htcondor-week-2022/djw8605",
        "jwks_uri":"https://sciauth.org/htcondor-week-2022/djw8605/oauth2/keys"
    }
    ```

    Start the pull request by clicking "Propose new file" at the bottom.  Then click "Create Pull Request"

2. Next, we will add the public keys to the pull request.  Under the pull request title, click on your branch.  Then, once again, click on "Add file" -> "Create new file".  This time, the path will be `username/oauth2/keys`.  The contents of the file will be the entire public key json above.  Be sure to include the starting and ending `{` and `}`.  Clicking on the commit at the bottom will add the new file to the pull request.

After both of the files have been created, the pull request will be reviewed by the maintainers and accepted.

Once your pull request has been merged, you can test the downloads with curl:

In [ ]:
curl https://sciauth.org/htcondor-week-2022/djw8605/.well-known/openid-configuration

In [ ]:
curl https://sciauth.org/htcondor-week-2022/djw8605/oauth2/keys

## Creating your first Token

Next, we will create your first token.

In [ ]:
scitokens-admin-create-token --keyfile=private.pem --key_id 2482 \
--issuer https://sciauth.org/htcondor-week-2022/djw8605 scope="compute.write:/ write:/ read:/"

## Submitting your first job

Lets submit our first job.  It will use the token you just created.  You will be submitting to a remote cluster that is configured to accept all issuers under https://sciauth.org/htcondor-week-2022/*

In [ ]:
condor_q

Lets copy our token to a known location

In [ ]:
scitokens-admin-create-token --keyfile=private.pem --key_id 2482  \
--issuer https://sciauth.org/htcondor-week-2022/djw8605 sub="dweitzel" scope="condor:READ condor:WRITE" > token

Save the token location to an environment variable for HTCondor to find

In [ ]:
export BEARER_TOKEN_FILE=$PWD/token

In [ ]:
condor_q

In [ ]:
condor_submit job.txt -spool

In [ ]:
condor_q

In [ ]:
condor_transfer_data <jobid>

In [ ]:
cat log

## Get a token from an issuer

We will be using [oidc-agent](https://indigo-dc.gitbook.io/oidc-agent/) to download a token to also submit a job.  This time, the issuer will instead by a public issuer, not a private key we generated.

Since oidc-agent uses a lot of interaction, we will be using the jupyter terminal.  Click on the blue "plus" sign on the left top, and select "Terminal" at the bottom.

The first step is to start the oidc-agent process.

In [ ]:
eval `oidc-agent-service use`

Next, setup the issuer configuration.  It will ask you several questions.  For scopes, select max.  You can just hit "enter" through the encryption password questions.

In [ ]:
oidc-gen issuer --issuer=https://lw-issuer.osgdev.chtc.io/scitokens-server/

Get the token and put it on the command line with the `oidc-token` command:

In [ ]:
oidc-token issuer

Save the new token to the `token` file.

In [ ]:
oidc-token issuer > token

In [ ]:
condor_q